In [1]:
import pandas as pd
import tldextract

In [4]:
train = pd.read_csv(r'train_data.csv',index_col = 0)
test = pd.read_csv(r'test_data.csv',index_col = 0)

In [45]:
train.isna().sum()

url          0
title        1
label        0
url_len      0
url_clean    0
url_len1     0
dtype: int64

In [35]:
test.isna().sum()

url      0
title    0
dtype: int64

In [46]:
train[train['title'].isna()]

,url,title,label,url_len,url_clean,url_len1
ID,,,,,,
78497,jpg-1.com,NaN,0,2,jpg-1,2


In [47]:
train.dropna(inplace = True)

# Чекним дизбаланс классов

In [7]:
train['label'].value_counts()/train.shape[0] * 100

label
0    87.646794
1    12.353206
Name: count, dtype: float64

Есть большой дизбаланс классов и с этим нужно будет что-то делать

# Обработаем данные
## Буду отдельно обрабатывать url и title

In [31]:
train['url_len1'] = train['url'].apply(lambda x: len(x.split('.')))

In [26]:
train['url_clean'] = train['url'].apply(lambda x: tldextract.extract(x).domain)

In [32]:
train[train['url_len'] == 1]

,url,title,label,url_len,url_clean,url_len1
ID,,,,,,
16537,login,Платформа Mail.Ru для бизнеса,0,1,login,1
17381,domains,Пользователь a.aborneva@oneproject.kz — Mail.r...,0,1,,1
18680,domains,Пользователь info@inex.studio — Mail.ru для би...,0,1,,1
21326,domains,Поиск пользователя «Селетков» — Mail.ru для би...,0,1,,1
22224,file:,РадиоКот :: Терморегулятор на термопаре К-типа,0,1,file,1
23616,login,Платформа Mail.Ru для бизнеса,0,1,login,1
24687,poslednie-novosti,Откройте для себя новый рынок сбыта благодаря,0,1,poslednie-novosti,1
26155,domains,Поиск пользователя «з» — Mail.ru для бизнеса,0,1,,1
27075,server2,104 Сменный блок для щетки Lilly латунь,0,1,server2,1


In [55]:
train[train['url_clean'] == '']

,url,title,label,url_len,url_clean,url_len1
ID,,,,,,
17381,domains,Пользователь a.aborneva@oneproject.kz — Mail.r...,0,1,,1
18680,domains,Пользователь info@inex.studio — Mail.ru для би...,0,1,,1
21326,domains,Поиск пользователя «Селетков» — Mail.ru для би...,0,1,,1
26155,domains,Поиск пользователя «з» — Mail.ru для бизнеса,0,1,,1
47933,domains,Поиск пользователя «фтв» — Mail.ru для бизнеса,0,1,,1
50481,domains,Поиск пользователя «Дягилева Лариса В» — Mail....,0,1,,1
53068,domains,Поиск пользователя «Галакти» — Mail.ru для биз...,0,1,,1
62055,domains,Поиск пользователя «yf» — Mail.ru для бизнеса,0,1,,1
64172,domains,Поиск пользователя «харитон» — Mail.ru для биз...,0,1,,1
